In [1]:
from pydantic import BaseModel
from openai import AsyncOpenAI


client = AsyncOpenAI(
    base_url="http://localhost:11434/v1/",
    # required but ignored
    api_key="ollama",
)


class TestModel(BaseModel):
    answer: str


results = []

i = client.beta.chat.completions.stream(
    model="llava",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    response_format=TestModel,
)

In [2]:
async with i as stream:
    async for chunk in stream:
        results.append(chunk)

In [3]:
results

[ChunkEvent(type='chunk', chunk=ChatCompletionChunk(id='chatcmpl-425', choices=[Choice(delta=ChoiceDelta(content='{', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1738171776, model='llava', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_ollama', usage=None), snapshot=ParsedChatCompletion[object](id='chatcmpl-425', choices=[ParsedChoice[object](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[object](content='{\n  "answer": ">Paris<" }\n ', refusal=None, role='assistantassistantassistantassistantassistantassistantassistantassistantassistantassistantassistantassistantassistantassistantassistantassistantassistant', audio=None, function_call=None, tool_calls=None, parsed=TestModel(answer='>Paris<')))], created=1738171776, model='llava', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=None)),
 ContentDeltaEvent(type='co

In [9]:
response.choices[0].message

ParsedChatCompletionMessage[TestModel](content='{ "answer": "Paris" } ', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=TestModel(answer='Paris'))

In [1]:
import asyncio
import logging
import aiohttp


async def retry_in():
    payload = {
        "messages": [{"role": "user", "content": "hello"}],
        "model": "llama3.2",
    }

    full_url = "http://localhost:11434/api/chat"  # <-- corrected

    headers = {
        "Authorization": "...",
        "Content-Type": "application/json",
    }

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(full_url, json=payload) as response:
                # Check the status first
                if response.status == 404:
                    # Typically means the endpoint doesn't exist
                    raise ValueError(
                        "Endpoint returned 404: check your route/path."
                    )

                # If the server actually returns JSON,
                # this will parse it. Otherwise, it may raise ContentTypeError
                response_json = await response.json()

                # If successful and no "error" key:
                if "error" not in response_json:
                    return response_json

        except asyncio.CancelledError:
            logging.warning("API call canceled by external request.")
            raise

        except aiohttp.ClientError as e:
            logging.error(f"API call failed: {e}")
            raise


result = await retry_in()
print(result)

ValueError: Endpoint returned 404: check your route/path.

In [ ]:
await retry_in()

In [1]:
messages = [
    {"role": "user", "content": "What is the capital of France?"},
]

In [2]:
from lionagi import iModel

from lionagi.service.endpoints.chat_completion import (
    EndPoint,
    ChatCompletionEndPoint,
)

from openai import AsyncOpenAI

client = AsyncOpenAI(
    base_url="http://localhost:11434/v1/",
    # required but ignored
    api_key="ollama",
)

response = await client.chat.completions.create(
    model="llava",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
)

print(response.choices[0].message.content)

 The capital of France is Paris. 


In [3]:
print(response.choices[0].message)

ChatCompletionMessage(content=' The capital of France is Paris. ', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [2]:
from lionagi import iModel

# gpt4o = iModel(
#     provider="openai",
#     model="gpt-4o",
#     # base_url="http://localhost:11434/api",
#     endpoint="chat",
#     # requires_api_key=False
# )

In [3]:
# api_call = gpt4o.create_api_calling(messages=messages)

In [5]:
class OllamaChatCompletionEndPoint(ChatCompletionEndPoint):

    async def _invoke(
        self,
    ): ...

    ...

In [6]:
CHAT_COMPLETION_CONFIG = {
    "provider": "ollama",
    "base_url": "http://localhost:11434/v1/api",
    "endpoint": "chat",
    "method": "post",
    "openai_compatible": True,
    "is_invokeable": True,
    "requires_tokens": True,
    "is_streamable": True,
    "requires_api_key": False,
    "required_kwargs": {
        "messages",
        "model",
    },
    "deprecated_kwargs": {
        "max_tokens",
        "function_call",
        "functions",
    },
    "optional_kwargs": {
        "store",
        "reasoning_effort",
        "metadata",
        "frequency_penalty",
        "logit_bias",
        "logprobs",
        "top_logprobs",
        "max_completion_tokens",
        "n",
        "modalities",
        "prediction",
        "audio",
        "presence_penalty",
        "response_format",
        "seed",
        "service_tier",
        "stop",
        "stream",
        "stream_options",
        "temperature",
        "top_p",
        "tools",
        "tool_choice",
        "parallel_tool_calls",
        "user",
    },
    "allowed_roles": ["user", "assistant", "system", "developer", "tool"],
}


ollama_endpoint = ChatCompletionEndPoint(CHAT_COMPLETION_CONFIG)

ollama = iModel(
    provider="ollama",
    model="deepseek-r1:1.5b",
    endpoint=ollama_endpoint,
    # invoke_with_endpoint=True,
)

In [7]:
api_call = ollama.create_api_calling(messages=messages)

In [8]:
api_call.payload

{'messages': [{'role': 'user', 'content': 'What is the capital of France?'}],
 'model': 'deepseek-r1:1.5b'}

In [9]:
result = await api_call.invoke()

ERROR:root:API call failed: 404, message='Attempt to decode JSON with unexpected mimetype: text/plain', url='http://localhost:11434/v1/api/chat'
